### 语言模型简介

语言模型的任务是预测每个句子在语言中出现的概率，把句子看成单词的序列，语言模型可以表示为一个计算 $p(w_1,w_2,w_3,\cdots,w_m)$ 的模型。那么怎么计算一个句子的概率呢？首先一个句子可以被看成是一个单词序列：$$S=(w_1,w_2,w_3,w_4,\cdots,w_m)$$ 其中 $m$ 为句子的长度。那么，它的概率可以表示为：
$$p(S)=p(w_1,w_2,w_3,\cdots,w_m)\\
=p(w_1)p(w_2|w_1)p(w_3|w_1,w_2)\cdots p(w_m|w_1,w_2,w_3,\cdots,w_{m-1})$$

这个式子参数比较多，为了估计这些参数的取值，常见的方法有 n-gram 模型、决策树、最大熵模型、条件随机场、神经网络模型等。先以 n-gram 为例，为了控制参数个数，n-gram 仅考虑前面的 $n-1$ 个单词：
$$p(S)=p(w_1,w_2,w_3,\cdots,w_m)=\prod^m_{i}p(w_i|w_{i-n+1},\cdots,w_{i-1})$$
n-gram 模型的参数一般采用最大似然估计方法计算：
$$p(w_i|w_{i-n+1},\cdots,w_{i-1})=\frac{C(w_{i-n+1},\cdots,w_{i-1},w_i)}{C(w_{i-n+1},\cdots,w_{i-1})}$$
其中，$C(X)$ 表示单词序列 $X$ 在训练语料中出现的次数。

### 语言模型的评价方法

语言模型效果好坏的常用评价指标是复杂度（perplexity）。在测试集上得到的 perplexity 越低，说明建模的效果要好。perplexity 的计算公式为：
$$perplexity(S)=p(w_1,w_2,w_3,\cdots,w_m)^{-1/m}\\
=\sqrt[m]{\frac{1}{p(w_1,w_2,w_3,\cdots,w_m)}}\\
=\sqrt[m]{\prod^m_{i=1}\frac{1}{p(w_i|w_1,\cdots,w_{i-1})}}$$
目前在 PTB 数据集上最好的语言模型 perplexity 为 47.7，也就是说，平均情况下，该模型预测下一个词时，有 47.7 个词等可能地可以作为下一个词的选择。

在训练中，通常采用 perlexity 的对数形式：
$$log(perplexity(S))=-\frac{1}{m}\sum^m_{i=1}log p(w_i|w_1,\cdots,w_{i-1})$$

### 神经语言模型

n-gram 只考虑前 $n$ 个单词的信息（通常 $n\leq4$），这就对语言模型的能力造成了限制。相比之下，RNN 可以将任意长度的上文信息存储在隐藏状态中，因此使用 RNN 作为语言模型有天然的优势。

#### PTB 数据集的预处理

从 Tomas Mikolov 网站上下载 PTB 数据，有测试集、训练集和验证集三个文件，文件已经过预处理，相邻单词之间用空格隔开。共包含 9998 个单词词汇，加上特殊符合 <unk> 和语句结束标记符（换行符），一共是 10000 个词汇。下面展示了训练数据的一行：
```
mr. <unk> is chairman of <unk> n.v. the dutch publishing group
```
为了将文本转化为可读入的单词序列，需要将这 10000 个词汇编号：

In [1]:
import collections
from operator import itemgetter

RAW_DATA = 'data/ptb.train.txt'
VOCAB_OUTPUT = 'data/ptb.vocab'

counter = collections.Counter()
with open(RAW_DATA, 'r', encoding='utf-8') as f:
    for line in f:
        for word in line.strip().split():
            counter[word] += 1
sorted_word_to_cnt = sorted(counter.items(), key=itemgetter(1),reverse=True)
sorted_words = [x[0] for x in sorted_word_to_cnt]
sorted_words = ['<eos>'] + sorted_words
with open(VOCAB_OUTPUT, 'w', encoding='utf-8') as output:
    for word in sorted_words:
        output.write(word + '\n')

有了词汇表之后，再将训练文件、测试文件等根据词汇文件转化为单词编号，每个单词的编号就是它在词汇文件中的行号。

In [2]:
RAW_DATA = 'data/ptb.train.txt'
OUTPUT_DATA = 'data/ptb.train'

with open(VOCAB_OUTPUT, 'r', encoding='utf-8') as f_vocab:
    vocab = [w.strip() for w in f_vocab.readlines()]
word_to_id = {k: v for (k, v) in zip(vocab, range(len(vocab)))}

def get_id(word):
    return word_to_id[word] if word in word_to_id.keys() else word_to_id['<unk>']

fin = open(RAW_DATA, 'r', encoding='utf-8')
fout = open(OUTPUT_DATA, 'w', encoding='utf-8')

for line in fin:
    words = line.strip().split() + ['<eos>']
    out_line = ' '.join([str(get_id(w)) for w in words]) + '\n'
    fout.write(out_line)

fin.close()
fout.close()

#### PTB 数据的 batching 方法

为了利用计算时的并行能力，batching 的方案为：先将整个文档切分成若干连续段落，再让 batch 中的每一个位置负责其中一段。

In [3]:
import numpy as np
import tensorflow as tf

TRAIN_DATA = 'data/ptb.train'
TRAIN_BATCH_SIZE = 20
TRAIN_NUM_SIZE = 35

def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as fin:
        id_string = ' '.join([line.strip() for line in fin.readlines()])
    id_list = [int(w) for w in id_string.split()]
    return id_list
    
def make_batches(id_list, batch_size, num_steps):
    num_batches = (len(id_list) - 1) // (batch_size * num_steps)
    data = np.array(id_list[: num_batches * batch_size * num_steps])
    data = np.reshape(data, [batch_size, num_batches * num_steps])
    data_batches = np.split(data, num_batches, axis=1)
    
    label = np.array(id_list[1: num_batches * batch_size * num_steps + 1])
    label = np.reshape(data, [batch_size, num_batches * num_steps])
    label_batches = np.split(label, num_batches, axis=1)
    
    return list(zip(data_batches, label_batches))

#### 基于 RNN 的神经语言模型

神经语言模型重要的两个层是词向量层和 softmax 层。

**词向量层**

词向量就是将词汇表嵌入到一个固定维度的实数空间里，词向量主要有两大作用：1）降低输入的维度；2）增加语义信息。

假设词向量的维度是 EMB_SIZE，词汇表的大小是 VOCAB_SIZE，那么所有单词的词向量可以放入大小为 VOCAB_SIZE * EMB_SIZE 的矩阵内。在读取词向量时，可以调用 tf.nn.embedding_lookup 方法。

In [ ]:
embedding = tf.get_variable('embedding', [VOCAB_SIZE, EMB_SIZE])

# 输出的矩阵比输入数据多了一个维度，新增维度的大小为 EMB_SIZE，
# 在语言模型中，一般 input_data 的维度是 batch_size * num_steps，
# 而输出的 input_embedding 的维度为 batch_size * num_steps * EMB_SIZE
input_embedding = tf.nn.embedding_lookup(embedding, input_data)

**Softmax 层**

Softmax 层的作用是将 RNN 的输出转化为一个单词表中每个单词的输出概率，主要有两个步骤：

1）使用线性映射将 RNN 的输出映射为一个词汇表大小维度的向量，这一步的输出叫做 logits。

In [ ]:
weight = tf.get_variable("weight", [HIDDEN_SIZE, VOCAB_SIZE])
bias = tf.get_variable("bias", [VOCAB_SIZE])
# output 是 RNN 的输出，其维度是 [batch_size * num_steps, HIDDEN_SIE]
logits = tf.nn.bias_add(tf.matmul(output, weight), bias)  # [batch_size * num_steps, VOCAB_SIZE]

2）将 logits 进行 softmax 操作。

In [ ]:
probs = tf.nn.softmax(logits)

模型训练通常不关心具体取值，可以直接调用 tf.nn.sparse_softmax_cross_entropy_with_logits 方法直接计算 log perplexity。

In [ ]:
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=tf.reshape(self.targets, [-1]), logits=logits)

共享词向量层和 softmax 层的参数，不仅能大幅度减少参数向量，还能提高模型效果。

In [4]:
import numpy as np
import tensorflow as tf

In [5]:
TRAIN_DATA = 'data/ptb.train'  # 训练集
EVAL_DATA = 'data/ptb.valid'  # 验证集
TEST_DATA = 'data/ptb.test'  # 测试集
HIDDEN_SIZE = 300  # 隐藏层大小
NUM_LAYERS = 2  # lstm 的层数
VOCAB_SIZE = 10000  # 词典规模
TRAIN_BATCH_SIZE = 20  # 训练集 batch 大小
TRAIN_NUM_STEP = 35  # 训练数据截断长度

EVAL_BATCH_SIZE = 1  # 测试数据 batch 的大小
EVAL_NUM_STEP = 1  # 测试数据截断长度
NUM_EPOCH = 5  # 使用训练数据的轮数
LSTM_KEEP_PROB = 0.9  # LSTM 节点不被 dropout 的概率
EMBEDDING_KEEP_PROB = 0.9  # 词向量不被 dropout 的概率
MAX_GRAD_NORM = 5  # 用于控制梯度膨胀的梯度大小上限
SHARE_EMB_AND_SOFTMAX = True  # 在 softmax 层和词向量层之间共享参数

In [16]:
class PTBModel(object):
    def __init__(self, is_training, batch_size, num_steps):
        self.batch_size = batch_size
        self.num_steps = num_steps
        
        # 输入和预期输出，大小都是 [batch_size, num_steps]
        self.input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
        self.targets = tf.placeholder(tf.int32, [batch_size, num_steps])
        
        # 定义使用 LSTM 结构为循环体结构且使用 dropout 的深层 RNN
        dropout_keep_prob = LSTM_KEEP_PROB if is_training else 1.0
        lstm_cells = [
            tf.nn.rnn_cell.DropoutWrapper(
                tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE), output_keep_prob=dropout_keep_prob)
            for _ in range(NUM_LAYERS)
        ]
        # 使用 MultiRNNCell 构建深层循环网络
        cell = tf.nn.rnn_cell.MultiRNNCell(lstm_cells)
        
        # 初始化最初的状态，即全零向量，只在每个 epoch 初始化第一个 batch 时使用
        # 因为有两层 LSTM，每层都有一个 initial_state，大小为 [batch, hidden_size]，后面的 state 也是如此。
        self.initial_state = cell.zero_state(batch_size, tf.float32)
        
        # 词向量矩阵
        embedding = tf.get_variable('embedding', [VOCAB_SIZE, HIDDEN_SIZE])
        
        # 将输入单词转化为词向量
        inputs = tf.nn.embedding_lookup(embedding, self.input_data)  # [batch_size, num_steps, HIDDEN_SIZE]
        
        if is_training:
            inputs = tf.nn.dropout(inputs, EMBEDDING_KEEP_PROB)
        
        # 定义输出列表，这里先将不同时刻 LSTM 结构的输出收集起来，再一起提供给 softmax 层
        outputs = []
        state = self.initial_state
        with tf.variable_scope('RNN'):
            for time_step in range(num_steps):
                if time_step > 0:
                    tf.get_variable_scope().reuse_variables()
                cell_output, state = cell(inputs[:, time_step, :], state)  # cell_output: [batch_size, hidden_size]
                outputs.append(cell_output)

        # outputs 是 num_steps 个 [batch_size, hidden_size]
        # tf.concat(outputs, 1) 是按第二个维度拼接，拼成 [batch, hidden * num_steps]
        # 然后 reshape 成 [batch * num_steps, hidden_size]
        output = tf.reshape(tf.concat(outputs, 1), [-1, HIDDEN_SIZE])  # [batch_size * num_steps, hidden_size]
        
        if SHARE_EMB_AND_SOFTMAX:
            weight = tf.transpose(embedding)
        else:
            weight = tf.get_variable('weight', [HIDDEN_SIZE, VOCAB_SIZE])
        bias = tf.get_variable('bias', [VOCAB_SIZE])
        logits = tf.matmul(output, weight) + bias  # [batch * num_steps, vocab_size]
        
        # labels 的维度为 [batch * num_steps]
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=tf.reshape(self.targets, [-1]),
            logits=logits)
        self.cost = tf.reduce_sum(loss) / batch_size
        self.final_state = state
        
        if not is_training:
            return
        
        trainable_variables = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(
            tf.gradients(self.cost, trainable_variables), MAX_GRAD_NORM)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0)
        self.train_op = optimizer.apply_gradients(
            zip(grads, trainable_variables))

In [17]:
def run_epoch(session, model, batches, train_op, output_log, step):
    total_costs = 0.0
    iters = 0
    state = session.run(model.initial_state)
    for x, y in batches:
        cost, state, _ = session.run(
            [model.cost, model.final_state, train_op],
            {model.input_data: x, model.targets: y,
            model.initial_state: state})
        total_costs += cost
        iters += model.num_steps
        
        if output_log and step % 100 == 0:
            print('After %d steps, perplexity is %.3f' % (step, np.exp(total_costs / iters)))
            
        step += 1
        
    return step, np.exp(total_costs / iters)

In [18]:
def main():
    initializer = tf.random_uniform_initializer(-0.05, 0.05)
    
    with tf.variable_scope('language_model',
                           reuse=None, initializer=initializer):
        train_model = PTBModel(True, TRAIN_BATCH_SIZE, TRAIN_NUM_STEP)
        
    with tf.variable_scope('language_model',
                           reuse=True, initializer=initializer):
        eval_model = PTBModel(False, EVAL_BATCH_SIZE, EVAL_NUM_STEP)

    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        train_batches = make_batches(
            read_data(TRAIN_DATA), TRAIN_BATCH_SIZE, TRAIN_NUM_STEP)
        eval_batches = make_batches(
            read_data(EVAL_DATA), EVAL_BATCH_SIZE, EVAL_NUM_STEP)
        test_batches = make_batches(
            read_data(TEST_DATA), EVAL_BATCH_SIZE, EVAL_NUM_STEP)

        step = 0
        for i in range(NUM_EPOCH):
            print('In iteration: %d' % (i + 1))
            step, train_pplx = run_epoch(sess, train_model, train_batches, train_model.train_op, True, step)
            print('Epoch: %d Train Perplexity: %.3f' % (i+1, train_pplx))

            _, eval_pplx = run_epoch(sess, eval_model, eval_batches, tf.no_op(), False, 0)
            print('Epoch: %d Eval Perplexity: %.3f' % (i+1, eval_pplx))

        _, test_pplx = run_epoch(sess, eval_model, test_batches, tf.no_op(), False, 0)
        print('Test Perplexity: %.3f' % test_pplx)

In [ ]:
if __name__ == '__main__':
    main()